<a href="https://colab.research.google.com/github/Stdunson/InsectDetectionProject/blob/main/InsectThings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Steps for Bioscan Script:
1. Download data
2. Remove the DNA and BIN data, remove all where family is not present
3. Load ResNet
4. Split Sets
5. Train, Validate
6. Test and Gather Data

# Steps for Insect Foundation Script:
1. Download data
2. Parse JSON File, remove all where family is not present
3. Load ResNet
4. Split Sets
5. Train, Validate
6. Test and Gather Data

#Inconsistencies
1. Not every data picture has every taxonomic level, so there has to be a way to filter that during gathering. The lowest level that every image consistently has is order, which is really high. We're gonna focus on family-level
2. One dataset is JSON and the other is raw images. Shouldn't matter too much buecause I'm gonna have two different training algorithms

